In [826]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Pré-processamento #

### Base e encoder manual ###

In [20]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [21]:
warnings.filterwarnings("ignore")

In [22]:
df_encod_manual = pd.DataFrame.copy(df)

In [23]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [24]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [25]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [26]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [27]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [28]:
previsores_label = df.iloc[:, 0:11].values

In [29]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [30]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [31]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [32]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [33]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [34]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [829]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# **Previsores** #
<span style="font-size: 15px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# Support Vector Machine #
<span style="font-size: small;">
<li> O algoritmo SVM (Máquina de Vetores de Suporte) é um método de aprendizado de máquina supervisionado utilizado para classificação e regressão.</br>
<li> Ele é usado para encontrar o hiperplano de separação ótimo que melhor divide as classes no espaço de características.</br>
<li> É amplamente utilizado em problemas de classificação, como reconhecimento de imagens, detecção de spam, diagnóstico médico, entre outros.</br>
<li> Descreve como encontrar o hiperplano que maximiza a margem entre as classes.</br>
<li> Assume a independência entre os atributos, dado o valor da classe.</br>
<li> Calcula as probabilidades a priori de cada classe e as probabilidades condicionais de cada atributo dado cada classe.</br>
<li> Para uma nova instância, calcula a probabilidade de pertencer a cada classe com base nos atributos e atribui a classe com a maior probabilidade.</br>


<strong>Hiperparâmetros Principais</strong>

- **C**: Penalidade por classificação errada. Controla a rigidez da margem de decisão.
- **Tipo de Kernel**: Escolha entre linear, polinomial, RBF (Radial Basis Function) e sigmoide.
- **Gamma**: Parâmetro para kernels não lineares que afeta a influência de cada exemplo de treinamento.
- **Coeficiente de Regularização (coef0)**: Afeta a influência dos termos de grau mais alto nos kernels polinomiais e sigmoide.
- **Tolerância de Erro**: Define a tolerância para a otimização do modelo.

</span>

## Criando e treinando ##

In [860]:
svm = SVC(
    kernel = 'linear', random_state = 5, C = 2
)
svm.fit(x_train, y_train);

## Previsões ##

In [861]:
previsoes_test = svm.predict(x_test)

In [862]:
previsoes_train = svm.predict(x_train)

### Métricas ###

#### Dados de teste ####
<span style="font-size: 15px;">Algoritmo avaliado em um conjunto separado de <strong>dados que não foram vistos durante o treinamento</strong></span>

In [863]:
accuracy_test = accuracy_score(y_test, previsoes_test) * 100.0
print("Acurácia dos dados de teste: %.2f%%" % accuracy_test)

Acurácia dos dados de teste: 85.87%


In [864]:
confMat_test = confusion_matrix(y_test, previsoes_test)
print("Matriz de confusão dos dados de teste: \n", confMat_test)

Matriz de confusão dos dados de teste: 
 [[102  19]
 [ 20 135]]


In [865]:
report_test = classification_report(y_test, previsoes_test)
print("Relatório dos dados de teste: \n", report_test)

Relatório dos dados de teste: 
               precision    recall  f1-score   support

           0       0.84      0.84      0.84       121
           1       0.88      0.87      0.87       155

    accuracy                           0.86       276
   macro avg       0.86      0.86      0.86       276
weighted avg       0.86      0.86      0.86       276



#### Dados de treino ####
<span style="font-size: 15px;">Algoritmo avaliado nos <strong>mesmos dados</strong> que foram utilizados para <strong>treiná-lo.</strong></span>

In [866]:
accuracy_train = accuracy_score(y_train, previsoes_train) * 100.0
print("Acurácia dos dados de treino: %.2f%%" % accuracy_train)

Acurácia dos dados de treino: 86.74%


In [867]:
confMat_train = confusion_matrix(y_train, previsoes_train)
print("Matriz de confusão dos dados de treino: \n", confMat_train)

Matriz de confusão dos dados de treino: 
 [[238  51]
 [ 34 318]]


In [868]:
report_train = classification_report(y_train, previsoes_train)
print("Relatório dos dados de treino: \n", report_train)

Relatório dos dados de treino: 
               precision    recall  f1-score   support

           0       0.88      0.82      0.85       289
           1       0.86      0.90      0.88       352

    accuracy                           0.87       641
   macro avg       0.87      0.86      0.87       641
weighted avg       0.87      0.87      0.87       641



### Validação Cruzada ###

In [869]:
kfold = KFold(
    n_splits = 30, shuffle = True, random_state = 5 
)

In [870]:
modelo = SVC(
    kernel = 'linear', random_state = 5, C = 2
)
resultado = cross_val_score(modelo, previsoresHot_esc, alvo, cv = kfold)

In [871]:
print("Acurácia média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia média: 85.71%
